In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

import librosa
import librosa.display
import IPython.display as ipd


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
data=pd.read_csv('Data/features_3_sec.csv')
data = data.iloc[:,2:]
data.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,1.080790e+06,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,6.722448e+05,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,7.907127e+05,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,9.216524e+05,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,6.102111e+05,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [4]:
y = data['label']
X = data.drop(['label','perceptr_var',"perceptr_mean"],axis=1)

In [5]:
y

0       blues
1       blues
2       blues
3       blues
4       blues
        ...  
9985     rock
9986     rock
9987     rock
9988     rock
9989     rock
Name: label, Length: 9990, dtype: object

In [6]:
X

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,1.080790e+06,...,-2.853603,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767
1,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,6.722448e+05,...,4.074709,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081
2,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,7.907127e+05,...,4.806280,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122
3,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,9.216524e+05,...,-1.359111,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678
4,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,6.102111e+05,...,2.092937,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,3015.559458,8.479527e+05,...,5.773784,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969
9986,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,3746.694524,1.170890e+06,...,2.074155,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033
9987,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,2442.362154,2.602871e+06,...,-1.005473,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990
9988,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,4313.266226,4.968878e+05,...,4.123402,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750


In [7]:
scaler = MinMaxScaler()
scaler.fit(X)
scaled_features=scaler.transform(X)

In [8]:
scaled_features

array([[0.3553988 , 0.71675699, 0.29313326, ..., 0.02679679, 0.50674637,
        0.04778078],
       [0.36732176, 0.67034733, 0.25303986, ..., 0.08241395, 0.59302914,
        0.06554772],
       [0.37315856, 0.72806736, 0.29675262, ..., 0.0438406 , 0.5462638 ,
        0.03606165],
       ...,
       [0.37419556, 0.69754793, 0.11650327, ..., 0.03992147, 0.50593726,
        0.04538043],
       [0.43653695, 0.65774457, 0.1482664 , ..., 0.01910101, 0.51990048,
        0.01373955],
       [0.40815099, 0.67614332, 0.11224896, ..., 0.03194326, 0.46137225,
        0.03454808]])

In [9]:
scaled_features.shape

(9990, 55)

In [10]:
data1=pd.DataFrame(scaled_features,columns=X.columns)
data1

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.355399,0.716757,0.293133,0.107955,0.262173,0.034784,0.459205,0.094130,0.346153,0.083164,...,0.363613,0.056198,0.397172,0.066062,0.371828,0.055344,0.380831,0.026797,0.506746,0.047781
1,0.367322,0.670347,0.253040,0.044447,0.270969,0.018716,0.470831,0.052261,0.363722,0.051694,...,0.468596,0.092912,0.351681,0.074001,0.362068,0.076365,0.418452,0.082414,0.593029,0.065548
2,0.373159,0.728067,0.296753,0.141663,0.265293,0.023073,0.494051,0.059922,0.378215,0.060820,...,0.479681,0.096704,0.420979,0.050639,0.400536,0.067509,0.433742,0.043841,0.546264,0.036062
3,0.399349,0.677066,0.298024,0.075042,0.238427,0.023187,0.455246,0.066234,0.329587,0.070906,...,0.386258,0.067995,0.387474,0.050617,0.380430,0.050030,0.405824,0.041898,0.562204,0.034873
4,0.355668,0.689113,0.322308,0.052149,0.233460,0.016451,0.451651,0.047830,0.318453,0.046916,...,0.438567,0.042500,0.460314,0.083860,0.388590,0.076524,0.409019,0.020763,0.517913,0.031713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,0.376757,0.617031,0.111106,0.002984,0.206932,0.034101,0.380041,0.068680,0.266983,0.065229,...,0.494342,0.060299,0.302553,0.069546,0.285713,0.044089,0.371046,0.040116,0.536263,0.042502
9986,0.413243,0.637013,0.128944,0.002687,0.277274,0.058465,0.438552,0.079860,0.349793,0.090105,...,0.438282,0.045545,0.249505,0.122777,0.305915,0.081395,0.263338,0.052963,0.516365,0.020232
9987,0.374196,0.697548,0.116503,0.021500,0.176099,0.138139,0.331163,0.111494,0.202062,0.200409,...,0.391617,0.112660,0.408760,0.038190,0.442668,0.035995,0.496892,0.039921,0.505937,0.045380
9988,0.436537,0.657745,0.148266,0.009825,0.324968,0.042367,0.473422,0.017567,0.413964,0.038187,...,0.469334,0.039551,0.362864,0.029532,0.471308,0.028749,0.506738,0.019101,0.519900,0.013740


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
X_train

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
5857,0.314395,0.080114,0.055205,0.000320,4343.018516,3.196191e+05,3416.882361,42879.144644,8881.783729,8.602397e+05,...,-2.881580,47.165234,-0.351201,49.027210,-1.800253,133.907181,13.181825,118.548889,3.427079,60.419411
3686,0.371759,0.080126,0.109284,0.001624,2137.029492,2.995402e+05,2257.282062,35521.007658,4791.551795,7.173886e+05,...,1.252087,20.650494,-7.956186,42.599293,5.827929,31.306114,-4.587831,32.225166,3.170006,37.735046
9930,0.375269,0.071575,0.118706,0.000257,2642.083580,3.891437e+04,2140.226902,22323.851055,4976.571702,2.888314e+05,...,2.704906,29.598747,-15.056197,18.042007,2.258145,24.990200,-10.485905,30.187914,-1.484748,23.793638
5214,0.217405,0.091227,0.038851,0.000965,1869.616044,4.976761e+05,2137.118423,199643.978962,4035.736366,3.048451e+06,...,20.371504,41.109505,16.007914,165.127777,13.958391,316.022369,12.030808,149.245941,3.305885,82.383461
3761,0.334128,0.089396,0.142549,0.002279,2239.433761,2.050969e+05,2191.172884,75308.816462,4620.114370,1.054409e+06,...,4.252404,49.965294,-1.731071,28.855803,5.747112,50.043049,-6.996763,67.405800,0.158898,55.908955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,0.267750,0.095907,0.097862,0.003840,2383.639867,4.895087e+05,2562.185167,362534.742879,5218.406137,3.237327e+06,...,-5.887548,31.795132,-4.608271,35.679741,-4.374666,25.779526,-3.147118,31.087011,-6.567071,63.435112
5191,0.248258,0.081754,0.041260,0.000109,1393.173520,5.375639e+04,1860.369053,40230.018073,2993.198054,4.600809e+05,...,7.150734,45.488049,-3.965447,14.019684,1.638268,16.482351,1.450863,30.808023,15.290517,134.934906
5390,0.240019,0.093735,0.083758,0.002298,1204.024837,1.078589e+05,1388.025813,24365.151773,2453.625676,5.744058e+05,...,-2.973363,77.487785,2.630663,42.383427,1.302412,55.033031,1.615667,48.945679,2.049629,95.770294
860,0.365543,0.092261,0.108228,0.002294,1511.812830,1.268170e+06,1781.554997,279045.135831,2899.280161,4.390229e+06,...,7.490125,72.863556,-8.287215,133.220337,0.265637,41.878937,-4.613067,66.480179,-1.747772,27.914318


In [13]:
X_test

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
4022,0.404232,0.096884,0.090223,0.000638,1416.001997,465268.853752,1973.083395,185535.919214,3098.462290,2.805922e+06,...,4.851411,55.957241,-4.883339,53.092667,1.034924,20.556194,-4.238788,31.233484,-2.934530,34.166332
5412,0.243589,0.084304,0.090146,0.000930,1206.214018,48883.472876,1523.543854,24788.400357,2516.651705,5.796811e+05,...,-4.166718,25.465181,-4.751444,25.725513,-8.050362,18.618586,-9.862431,53.661839,-11.109019,12.743995
487,0.355097,0.086465,0.130576,0.001197,2293.981066,184298.037527,2181.331825,108973.121405,4714.280724,9.991080e+05,...,1.277914,65.851204,-4.983762,28.220079,-4.208559,46.466846,-8.962951,47.620399,-3.131865,34.480564
39,0.387297,0.092417,0.149871,0.005382,1061.389626,128482.413202,1665.487663,123692.389644,2089.548903,1.090108e+06,...,2.606227,61.515476,-7.273301,54.165520,0.735078,30.324575,-2.341069,49.825420,-6.121044,18.358120
6795,0.419387,0.079487,0.153464,0.001511,2739.278464,81717.413705,2410.267381,12507.457839,5818.437124,1.978264e+05,...,11.295089,28.295105,-3.922399,25.422867,3.707712,34.378510,-9.237077,65.149940,1.359221,57.023216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,0.345927,0.086218,0.045433,0.000071,1032.931007,9044.241152,1200.481124,2419.441461,1537.222243,3.623162e+04,...,-9.387864,31.557072,8.398466,27.667841,14.495088,23.355673,3.106972,54.574383,-0.279830,21.384857
7905,0.422266,0.098932,0.255165,0.011359,3517.987701,658421.127209,3248.575897,117811.287197,7755.514386,1.839925e+06,...,-3.855971,41.022110,-7.146739,32.625473,-2.940797,52.929779,-4.194828,78.605034,-2.255957,72.413223
6113,0.579352,0.052333,0.109304,0.000085,2916.971171,35773.228649,2165.956722,17303.496264,5332.200834,1.419968e+05,...,4.276198,20.069010,-9.720901,19.325178,3.899539,16.666124,-3.626257,24.900068,1.132221,25.351131
3532,0.416550,0.077638,0.195324,0.002525,2884.484100,502342.626998,2619.914987,106196.076239,6250.343487,1.414326e+06,...,4.268992,39.432400,-1.048392,46.652473,5.630126,32.986427,0.541846,79.514961,1.222449,36.466000


In [14]:
print(y_train)
print("\n\n")
print(y_test)

5857     jazz
3686    disco
9930     rock
5214     jazz
3761    disco
        ...  
5734     jazz
5191     jazz
5390     jazz
860     blues
7270      pop
Name: label, Length: 6993, dtype: object



4022       hiphop
5412         jazz
487         blues
39          blues
6795        metal
          ...    
1579    classical
7905          pop
6113        metal
3532        disco
409         blues
Name: label, Length: 2997, dtype: object


In [73]:
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.1, eval_metric='logloss')

In [74]:
xgb.fit(X_train, y_train)
preds = xgb.predict(X_test)

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [75]:
accuracy_score(y_test, preds)

0.8938938938938938

In [79]:
confusion_matrix(y_test, preds)

array([[270,   2,  19,   6,   0,  11,   4,   0,   2,   5],
       [  0, 300,   1,   0,   1,   4,   0,   0,   0,   2],
       [ 12,   0, 244,   5,   1,  12,   1,   2,   5,   4],
       [  1,   4,   3, 266,   2,   3,   4,  11,   2,   5],
       [  2,   1,   3,   6, 280,   2,   2,   7,   5,   3],
       [  2,  12,   4,   2,   0, 260,   0,   1,   3,   2],
       [  5,   0,   5,   2,   7,   0, 278,   0,   0,   6],
       [  0,   0,   4,   3,   2,   0,   0, 253,   3,   2],
       [  2,   2,   7,   5,   4,   2,   1,   3, 287,   3],
       [  5,   1,  12,  16,   3,   4,  12,   3,   3, 241]], dtype=int64)

In [80]:
pred_audio_cm = pd.read_csv("pred_audio_cm.csv")
pred_audio_cm.drop('Unnamed: 0',axis=1,inplace=True)
pred_audio_cm

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.309224,0.082895,0.143282,0.002155,2997.182631,732097.83995,2800.86838,168214.652343,6331.986711,2.639551e+06,...,9.96054,127.599106,1.229967,82.59022,5.445252,76.23788,1.485082,71.17988,7.496902,62.828804


In [81]:
pred=xgb.predict(pred_audio_cm)
pred

array(['disco'], dtype=object)

In [84]:
import pickle
file_name = "xgbmodel1"

# save
pickle.dump(xgb, open(file_name, "wb"))
